In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from prophet import Prophet
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# Load the dataset
df= pd.read_excel('/content/drive/MyDrive/Colab Notebooks/DG/forecasting_case_study.xlsx')
#df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DG/forecasting_case_study.csv')

df['date'] = pd.to_datetime(df['date'])

#df['Price Discount (%)'] = df['Price Discount (%)'].str.replace('%', '').astype(float)
#cols = df.select_dtypes(['numerical']).columns.tolist()
#cols



In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Product             1218 non-null   object        
 1   date                1218 non-null   datetime64[ns]
 2   Sales               1218 non-null   int64         
 3   Price Discount (%)  1218 non-null   float64       
 4   In-Store Promo      1218 non-null   int64         
 5   Catalogue Promo     1218 non-null   int64         
 6   Store End Promo     1218 non-null   int64         
 7   Google_Mobility     1218 non-null   float64       
 8   Covid_Flag          1218 non-null   int64         
 9   V_DAY               1218 non-null   int64         
 10  EASTER              1218 non-null   int64         
 11  CHRISTMAS           1218 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(8), object(1)
memory usage: 114.3+ KB


In [4]:
df['Price Discount (%)'] = df['Price Discount (%)'].astype(str)
df['Price Discount (%)'] = df['Price Discount (%)'].str.replace('%', '').astype(float)
df['Price Discount (%)'] = df['Price Discount (%)'].round(2)
df['Price Discount (%)'] = df['Price Discount (%)']*100


In [5]:
#df['Price Discount (%)']= df['Price Discount (%)'].str[:-1]

df['Price Discount (%)'] = df['Price Discount (%)'].replace('', np.nan).astype(float).fillna(0).astype(int)

In [6]:
df['Price Discount (%)']

0        0
1        0
2       17
3        0
4       17
        ..
1213    54
1214    52
1215    54
1216    44
1217    44
Name: Price Discount (%), Length: 1218, dtype: int64

In [7]:
df['Google_Mobility']= df['Google_Mobility'].round(2)

In [8]:
df.head(30)

,Product,date,Sales,Price Discount (%),In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
0,SKU1,2017-02-05,27750,0,0,0,0,0.0,0,0,0,0
1,SKU1,2017-02-12,29023,0,1,0,1,0.0,0,1,0,0
2,SKU1,2017-02-19,45630,17,0,0,0,0.0,0,0,0,0
3,SKU1,2017-02-26,26789,0,1,0,1,0.0,0,0,0,0
4,SKU1,2017-03-05,41999,17,0,0,0,0.0,0,0,0,0
5,SKU1,2017-03-12,29731,0,0,0,0,0.0,0,0,0,0
6,SKU1,2017-03-19,27365,0,1,0,0,0.0,0,0,0,0
7,SKU1,2017-03-26,27722,0,1,0,1,0.0,0,0,0,0
8,SKU1,2017-04-02,44339,17,1,0,0,0.0,0,0,0,0
9,SKU1,2017-04-09,54655,17,1,0,0,0.0,0,0,1,0


In [9]:
cols=['In-Store Promo','Catalogue Promo','Store End Promo','Covid_Flag','V_DAY','EASTER','CHRISTMAS']
for i in cols:

  df[i]=df[i].astype('category')
  print(df[i])

0       0
1       1
2       0
3       1
4       0
       ..
1213    0
1214    0
1215    1
1216    1
1217    0
Name: In-Store Promo, Length: 1218, dtype: category
Categories (2, int64): [0, 1]
0       0
1       0
2       0
3       0
4       0
       ..
1213    1
1214    1
1215    0
1216    0
1217    0
Name: Catalogue Promo, Length: 1218, dtype: category
Categories (2, int64): [0, 1]
0       0
1       1
2       0
3       1
4       0
       ..
1213    0
1214    0
1215    1
1216    1
1217    0
Name: Store End Promo, Length: 1218, dtype: category
Categories (2, int64): [0, 1]
0       0
1       0
2       0
3       0
4       0
       ..
1213    1
1214    1
1215    1
1216    1
1217    1
Name: Covid_Flag, Length: 1218, dtype: category
Categories (2, int64): [0, 1]
0       0
1       1
2       0
3       0
4       0
       ..
1213    0
1214    0
1215    0
1216    0
1217    0
Name: V_DAY, Length: 1218, dtype: category
Categories (2, int64): [0, 1]
0       0
1       0
2       0
3       0
4       0
 

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Product             1218 non-null   object        
 1   date                1218 non-null   datetime64[ns]
 2   Sales               1218 non-null   int64         
 3   Price Discount (%)  1218 non-null   int64         
 4   In-Store Promo      1218 non-null   category      
 5   Catalogue Promo     1218 non-null   category      
 6   Store End Promo     1218 non-null   category      
 7   Google_Mobility     1218 non-null   float64       
 8   Covid_Flag          1218 non-null   category      
 9   V_DAY               1218 non-null   category      
 10  EASTER              1218 non-null   category      
 11  CHRISTMAS           1218 non-null   category      
dtypes: category(7), datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 56.9+ KB


In [11]:
# Extract features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['week'] = df['date'].dt.week
df['day_of_week'] = df['date'].dt.dayofweek
for lag in [1, 2, 3]:
    df[f'sales_lag_{lag}'] = df.groupby('Product')['Sales'].shift(lag)
df['rolling_mean_3'] = df.groupby('Product')['Sales'].transform(lambda x: x.rolling(window=3).mean())
df['rolling_std_3'] = df.groupby('Product')['Sales'].transform(lambda x: x.rolling(window=3).std())

# Normalize
scaler = StandardScaler()
numerical_cols = ['Price Discount (%)', 'Google_Mobility', 'sales_lag_1', 'sales_lag_2', 'sales_lag_3', 'rolling_mean_3', 'rolling_std_3']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
df.dropna(inplace=True)

# Synthetic data generation using ARIMA and Decomposition
synthetic_data = []
skus = df['Product'].unique()
np.random.seed(42)

#N = len(sku_data)  # The length of the original series for each SKU

for sku in skus:
    sku_data = df[df['Product'] == sku]
    N = len(sku_data)  # The length of the original series for each SKU

    series = sku_data['Sales'].values

    # ARIMA based synthetic generation
    arima = ARIMA(series, order=(5,1,0))
    arima_fitted = arima.fit()
    #arima_forecast = arima_fitted.forecast(steps=len(series) + N - len(series))
    #synthetic_series_arima = np.concatenate([series, arima_forecast[:N - len(series)]])

    arima_forecast = arima_fitted.forecast(steps= len(series) +N - len(series))
    synthetic_series_arima = np.concatenate([series, arima_forecast[:N - len(series)]])

    # Choose decomposition model based on data
    if np.any(series <= 0):
        decompose_model = 'additive'
    else:
        decompose_model = 'multiplicative'

    # Decomposition based synthetic generation
    result = seasonal_decompose(series, model=decompose_model, extrapolate_trend='freq', period=12)
    residual = result.resid
    seasonal = result.seasonal
    trend = result.trend
    synthetic_series_decompose = (trend + np.random.choice(residual, size=N, replace=True) + seasonal)[:N]


    # Combine the synthetic series
    combined_synthetic_series = (synthetic_series_arima + synthetic_series_decompose) / 2

    #prophet_forecast = model.predict(future_data2)['yhat'].tail(len(test)).values

    # Append to synthetic data
    synthetic_data.append(pd.DataFrame({
        'Product': [sku] * N,
        'Sales': combined_synthetic_series,
        'date': np.tile(sku_data['date'].values, (int(N/len(series)), )),
        'year': np.tile(sku_data['year'].values, (int(N/len(series)), )),
        'month': np.tile(sku_data['month'].values, (int(N/len(series)), )),
        'week': np.tile(sku_data['week'].values, (int(N/len(series)), )),
        'day_of_week': np.tile(sku_data['day_of_week'].values, (int(N/len(series)), )),
    }))

synthetic_df = pd.concat(synthetic_data)
new_df = pd.concat([df, synthetic_df])

In [12]:
import numpy as np

def mean_absolute_percentage_error(y_test, ensemble_forecast):
    """
    Calculate Mean Absolute Percentage Error (MAPE).

    Parameters:
    y_test (array-like): True target values.
    ensemble_forecast (array-like): Predicted target values.

    Returns:
    float: MAPE score.
    """
    y_true, y_pred = np.array(y_test), np.array(ensemble_forecast)
    return np.mean(np.abs((y_test - ensemble_forecast) / y_test)) * 100

In [13]:
!pip install prophet
! pip install fbprophet --upgrade
! pip install tensorflow --upgrade


  Using cached fbprophet-0.7.1.tar.gz (64 kB)
  Preparing metadata (setup.py) ... done
  Using cached cmdstanpy-0.9.5-py3-none-any.whl (37 kB)
  Using cached pystan-3.7.0-py3-none-any.whl (13 kB)
  Using cached setuptools_git-1.2-py2.py3-none-any.whl (10 kB)
  Using cached clikit-0.6.2-py2.py3-none-any.whl (91 kB)
  Using cached httpstan-4.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (44.4 MB)
  Using cached pysimdjson-5.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.8 MB)
  Using cached crashtest-0.3.1-py3-none-any.whl (7.0 kB)
  Using cached pastel-0.2.1-py2.py3-none-any.whl (6.0 kB)
  Using cached pylev-1.4.0-py2.py3-none-any.whl (6.1 kB)
  Using cached marshmallow-3.20.1-py3-none-any.whl (49 kB)
  Using cached webargs-8.3.0-py3-none-any.whl (31 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and

In [14]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from prophet import Prophet
import numpy as np
import warnings
try:
  from bayes_opt import BayesianOptimization
except:
  !pip install bayesian-optimization
  from bayes_opt import BayesianOptimization

from sklearn.metrics import mean_squared_error

warnings.filterwarnings('ignore')  # Suppress warnings

def create_features(data):
    data['lag_1'] = data['Sales'].shift(1)
    data['lag_2'] = data['Sales'].shift(2)
    data['lag_3'] = data['Sales'].shift(3)
    data['rolling_mean'] = data['Sales'].rolling(window=3).mean()
    return data

def arima_evaluate(p, d, q):
    try:
        model = ARIMA(y_train, order=(int(p), int(d), int(q)))
        model_fitted = model.fit()
        forecast = model_fitted.forecast(steps=len(test))
        return -mean_squared_error(y_test, forecast)
    except:
        return float('-inf')

def rfr_evaluate(n_estimators, max_depth):
    model = RandomForestRegressor(n_estimators=int(n_estimators), max_depth=int(max_depth))
    model.fit(X_train_scaled, y_train)
    predictions = model.predict(X_test_scaled)
    return -mean_squared_error(y_test, predictions)

def gbm_evaluate(n_estimators, learning_rate, max_depth):
    model = GradientBoostingRegressor(n_estimators=int(n_estimators), learning_rate=learning_rate, max_depth=int(max_depth))
    model.fit(X_train_scaled, y_train)
    predictions = model.predict(X_test_scaled)
    return -mean_squared_error(y_test, predictions)

arima_bounds = {
    'p': (0, 5),
    'd': (0, 3),
    'q': (0, 5)
}

rfr_bounds = {
    'n_estimators': (50, 500),
    'max_depth': (1, 10)
}

gbm_bounds = {
    'n_estimators': (50, 500),
    'learning_rate': (0.01, 0.8),
    'max_depth': (1, 10)
}

results = {}
ensemble_results = {}
skus = df['Product'].unique()
scaler = MinMaxScaler()

for sku in skus:
    print(f"Processing SKU: {sku}")

    sku_data = df[df['Product'] == sku]
    sku_data = create_features(sku_data)

    train = sku_data[sku_data['date'] <= '2020-09-30'].dropna()
    test = sku_data[sku_data['date'] > '2020-09-30']

    X_train = train.drop(['Sales', 'Product', 'date'], axis=1)
    y_train = train['Sales']
    X_test = test.drop(['Sales', 'Product', 'date'], axis=1)
    y_test = test['Sales']

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Handle NaNs
    X_train_scaled = np.nan_to_num(X_train_scaled)
    X_test_scaled = np.nan_to_num(X_test_scaled)

    # ARIMA Optimization
    optimizer = BayesianOptimization(arima_evaluate, arima_bounds, random_state=27)
    optimizer.maximize(init_points=5, n_iter=100)
    best_params = optimizer.max['params']
    arima = ARIMA(y_train, order=(int(best_params['p']), int(best_params['d']), int(best_params['q'])))
    arima_fitted = arima.fit()
    arima_forecast = arima_fitted.forecast(steps=len(test))

    # RandomForestRegressor Optimization
    optimizer = BayesianOptimization(rfr_evaluate, rfr_bounds, random_state=27)
    optimizer.maximize(init_points=5, n_iter=100)
    best_params = optimizer.max['params']
    rfr = RandomForestRegressor(n_estimators=int(best_params['n_estimators']), max_depth=int(best_params['max_depth']))
    rfr.fit(X_train_scaled, y_train)
    rfr_forecast = rfr.predict(X_test_scaled)

    # GradientBoostingRegressor Optimization
    optimizer = BayesianOptimization(gbm_evaluate, gbm_bounds, random_state=27)
    optimizer.maximize(init_points=5, n_iter=100)
    best_params = optimizer.max['params']
    gbm = GradientBoostingRegressor(n_estimators=int(best_params['n_estimators']), learning_rate=best_params['learning_rate'], max_depth=int(best_params['max_depth']))
    gbm.fit(X_train_scaled, y_train)
    gbm_forecast = gbm.predict(X_test_scaled)

    # Prophet
    try:
        prophet_data = pd.DataFrame({'ds': train['date'], 'y': y_train})
        model_prophet = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=True)
        model_prophet.fit(prophet_data)
        future = model_prophet.make_future_dataframe(periods=len(test))
        forecast_prophet = model_prophet.predict(future)
        prophet_forecast = forecast_prophet['yhat'][-len(test):].values
    except:
        prophet_forecast = [0] * len(test)



    #prophet_data  = train.rename(columns={'Sales' : 'y', 'date': 'ds'})
    #prophet_data['y_orig'] = prophet_data['y']
    #prophet_data['y'] = np.log(prophet_data['y'])

    #model_new = Prophet()
    #model_new.add_regressor('Price Discount (%)')
    #model_new.add_regressor('In-Store Promo')
    #model_new.add_regressor('Catalogue Promo')
    #model_new.add_regressor('Store End Promo')
    #model_new.add_regressor('Google_Mobility')
    #model_new.add_regressor('Covid_Flag')
    #model_new.add_regressor('Holiday_Flag')
    #model_new.add_regressor('V_DAY')
    #model_new.add_regressor('EASTER')
    #model_new.add_regressor('CHRISTMAS')

    #cutoff_date = pd.to_datetime('2020-11-22')
    #prophet_data['ds'] = pd.to_datetime(prophet_data['ds'])
    #prophet_data.loc[prophet_data['ds'] >= cutoff_date, 'y'] = 0

    #for column in prophet_data.columns:
    #data_type = prophet_data[column].dtype
    #print(f'Column "{column}" has data type: {data_type}')

    #if 'DateColumn1' in prophet_data.columns and 'DateColumn2' in prophet_data.columns:
        #print('Both DateColumn1 and DateColumn2 are present.')

   # else:
        #print('Timestamp columns are not present.')

    #prophet_data['ds'] = pd.to_datetime(prophet_data['ds'])

    #data_type = prophet_data['ds'].dtype
    #print(f"'ds' column has been converted to data type: {data_type}")

    #np.array(prophet_data['ds'], dtype=np.datetime64)

    #model = Prophet(weekly_seasonality=True)

    #prophet_data['ds'] = pd.to_datetime(prophet_data['ds'])

    #model = Prophet()

    #model.fit(prophet_data)

    #future_data2 = model.make_future_dataframe(periods=204, freq='W')

    #prophet_forecast = model.predict(future_data2)

    #future_data2 = model.make_future_dataframe(periods=204, freq='W')

    #prophet_forecast = model.predict(future_data2)['yhat'].tail(len(test)).values



    # Ensembling (Simple averaging)
    ensemble_forecast = (np.array(arima_forecast) + rfr_forecast + gbm_forecast + prophet_forecast) / 4
    mape_ensemble = mean_absolute_percentage_error(y_test, ensemble_forecast)  # Assuming mean_absolute_percentage_error function is defined elsewhere
    accuracy_ensemble = 1 - mape_ensemble

    ensemble_results[sku] = {
        'MAPE': mape_ensemble,
        'Accuracy': accuracy_ensemble
    }

print(ensemble_results)

Processing SKU: SKU1
|   iter    |  target   |     d     |     p     |     q     |
-------------------------------------------------------------
| 1         | -1.201e+0 | 1.277     | 4.073     | 3.677     |
| 2         | -9.242e+0 | 2.604     | 1.917     | 4.897     |
| 3         | -8.425e+0 | 2.68      | 1.049     | 3.709     |
| 4         | -1.146e+0 | 1.989     | 4.434     | 4.29      |
| 5         | -4.463e+0 | 2.248     | 4.351     | 0.9338    |
| 6         | -1.172e+0 | 0.0       | 0.0       | 5.0       |
| 7         | -1.24e+09 | 0.0       | 5.0       | 5.0       |
| 8         | -1.159e+0 | 0.0       | 0.0       | 1.532     |
| 9         | -1.174e+0 | 0.0       | 2.488     | 5.0       |
| 10        | -2.28e+12 | 3.0       | 0.0       | 0.0       |
| 11        | -1.172e+0 | 0.0       | 5.0       | 0.0       |
| 12        | -1.163e+0 | 0.0       | 2.457     | 1.96      |
| 13        | -8.83e+08 | 3.0       | 0.0       | 5.0       |
| 14        | -3.71e+09 | 3.0       | 5.0       |

DEBUG:cmdstanpy:input tempfile: /tmp/tmpc277953y/_sc5ik35.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpc277953y/d144wt_o.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=9546', 'data', 'file=/tmp/tmpc277953y/_sc5ik35.json', 'init=/tmp/tmpc277953y/d144wt_o.json', 'output', 'file=/tmp/tmpc277953y/prophet_modelti6120_l/prophet_model-20231007195352.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:53:52 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:53:52 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Processing SKU: SKU2
|   iter    |  target   |     d     |     p     |     q     |
-------------------------------------------------------------
| 1         | -1.157e+0 | 1.277     | 4.073     | 3.677     |
| 2         | -1.257e+0 | 2.604     | 1.917     | 4.897     |
| 3         | -1.288e+0 | 2.68      | 1.049     | 3.709     |
| 4         | -1.165e+0 | 1.989     | 4.434     | 4.29      |
| 5         | -1.495e+0 | 2.248     | 4.351     | 0.9338    |
| 6         | -1.183e+0 | 0.0       | 4.743     | 5.0       |
| 7         | -1.12e+08 | 0.0       | 2.316     | 4.907     |
| 8         | -1.256e+0 | 0.0       | 0.0       | 5.0       |
| 9         | -1.294e+0 | 0.0       | 0.0       | 0.0       |
| 10        | -1.518e+0 | 0.0       | 2.565     | 3.363     |
| 11        | -1.165e+0 | 1.208     | 4.424     | 4.322     |
| 12        | -1.557e+0 | 0.5206    | 3.214     | 5.0       |
| 13        | -1.267e+0 | 0.1197    | 4.404     | 1.526     |
| 14        | -1.12e+08 | 0.6976    | 2.473     |

DEBUG:cmdstanpy:input tempfile: /tmp/tmpc277953y/o0xvmze2.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpc277953y/2nhvfk67.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=68105', 'data', 'file=/tmp/tmpc277953y/o0xvmze2.json', 'init=/tmp/tmpc277953y/2nhvfk67.json', 'output', 'file=/tmp/tmpc277953y/prophet_modelsfba5a2h/prophet_model-20231007195822.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:58:22 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:58:22 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Processing SKU: SKU3
|   iter    |  target   |     d     |     p     |     q     |
-------------------------------------------------------------
| 1         | -2.219e+0 | 1.277     | 4.073     | 3.677     |
| 2         | -2.02e+09 | 2.604     | 1.917     | 4.897     |
| 3         | -2.023e+0 | 2.68      | 1.049     | 3.709     |
| 4         | -2.298e+0 | 1.989     | 4.434     | 4.29      |
| 5         | -1.042e+0 | 2.248     | 4.351     | 0.9338    |
| 6         | -8.816e+1 | 3.0       | 5.0       | 0.0       |
| 7         | -2.259e+0 | 1.764     | 3.912     | 4.031     |
| 8         | -2.009e+0 | 1.751     | 3.81      | 1.962     |
| 9         | -2.034e+0 | 0.8578    | 3.848     | 0.7258    |
| 10        | -4.281e+0 | 2.125     | 2.701     | 0.896     |
| 11        | -1.193e+0 | 3.0       | 2.699     | 3.129     |
| 12        | -2.452e+0 | 1.299     | 2.101     | 3.419     |
| 13        | -1.201e+0 | 2.897     | 4.742     | 2.929     |
| 14        | -2.023e+0 | 1.159     | 0.7546    |

...

Processing SKU: SKU5
|   iter    |  target   |     d     |     p     |     q     |
-------------------------------------------------------------
| 1         | -1.988e+0 | 1.277     | 4.073     | 3.677     |
| 2         | -9.405e+0 | 2.604     | 1.917     | 4.897     |
| 3         | -9.495e+0 | 2.68      | 1.049     | 3.709     |
| 4         | -2.247e+0 | 1.989     | 4.434     | 4.29      |
| 5         | -1.151e+0 | 2.248     | 4.351     | 0.9338    |
| 6         | -2.189e+0 | 0.0       | 0.2413    | 5.0       |
| 7         | -2.086e+0 | 0.0       | 0.0       | 1.409     |
| 8         | -2.045e+0 | 0.0       | 3.214     | 5.0       |
| 9         | -1.364e+0 | 3.0       | 0.0       | 5.0       |
| 10        | -2.112e+0 | 3.0       | 0.0       | 0.0       |
| 11        | -2.106e+0 | 0.0       | 5.0       | 5.0       |
| 12        | -1.599e+0 | 3.0       | 0.0       | 2.255     |
| 13        | -2.068e+0 | 0.0       | 1.742     | 3.125     |
| 14        | -2.078e+0 | 1.353     | 0.0       |

DEBUG:cmdstanpy:input tempfile: /tmp/tmpc277953y/1q60ichl.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpc277953y/a5g0tq6n.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=79931', 'data', 'file=/tmp/tmpc277953y/1q60ichl.json', 'init=/tmp/tmpc277953y/a5g0tq6n.json', 'output', 'file=/tmp/tmpc277953y/prophet_model4vz0rrcx/prophet_model-20231007201133.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
20:11:33 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
20:11:33 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Processing SKU: SKU6
|   iter    |  target   |     d     |     p     |     q     |
-------------------------------------------------------------
| 1         | -3.301e+0 | 1.277     | 4.073     | 3.677     |
| 2         | -4.031e+0 | 2.604     | 1.917     | 4.897     |
| 3         | -3.476e+0 | 2.68      | 1.049     | 3.709     |
| 4         | -3.237e+0 | 1.989     | 4.434     | 4.29      |
| 5         | -2.378e+0 | 2.248     | 4.351     | 0.9338    |
| 6         | -3.335e+1 | 3.0       | 3.222     | 0.0       |
| 7         | -3.723e+0 | 1.942     | 3.316     | 4.225     |
| 8         | -3.28e+09 | 1.867     | 4.964     | 2.105     |
| 9         | -3.248e+0 | 0.8167    | 5.0       | 0.7181    |
| 10        | -3.249e+0 | 0.2024    | 3.547     | 1.915     |
| 11        | -3.268e+0 | 0.5666    | 1.775     | 3.375     |
| 12        | -3.806e+0 | 1.022     | 0.3113    | 4.789     |
| 13        | -3.233e+0 | 0.04452   | 2.766     | 5.0       |
| 14        | -2.997e+0 | 0.0288    | 4.82      |

DEBUG:cmdstanpy:input tempfile: /tmp/tmpc277953y/w0tzw7uo.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpc277953y/aw8s14lj.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=99074', 'data', 'file=/tmp/tmpc277953y/w0tzw7uo.json', 'init=/tmp/tmpc277953y/aw8s14lj.json', 'output', 'file=/tmp/tmpc277953y/prophet_modeljgyyg8hy/prophet_model-20231007201539.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
20:15:39 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
20:15:39 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


{'SKU1': {'MAPE': inf, 'Accuracy': -inf}, 'SKU2': {'MAPE': inf, 'Accuracy': -inf}, 'SKU3': {'MAPE': inf, 'Accuracy': -inf}, 'SKU4': {'MAPE': inf, 'Accuracy': -inf}, 'SKU5': {'MAPE': inf, 'Accuracy': -inf}, 'SKU6': {'MAPE': 49.59008991100976, 'Accuracy': -48.59008991100976}}


In [15]:
def interpret_dynamic_results(results):
    interpretations = {}

    # Categorize SKUs
    normal_skus = [sku for sku, metrics in results.items() if 0 <= metrics['Accuracy'] <= 1]
    abnormal_skus = [sku for sku, metrics in results.items() if not 0 <= metrics['Accuracy'] <= 1]

    for sku, metrics in results.items():
        mape = metrics['MAPE']
        accuracy = metrics['Accuracy']
        interpretation = ""

        if sku in abnormal_skus:
            interpretation += (f"For {sku}, the MAPE value is alarmingly high, suggesting the forecast might be producing extreme values. "
                               f"The 'Accuracy' is outside the standard range of 0% to 100%, indicating significant forecast errors. "
                               "The large discrepancy in the model's forecast suggests potential issues like extreme values, outliers, or "
                               "a model not suited for this data. Immediate reevaluation is recommended for this SKU.")
        elif sku in normal_skus:
            interpretation += (f"For {sku}, the MAPE is {mape:.4f}, indicating the predictions are, on average, off by about {mape*100:.2f}%. "
                               f"The forecast accuracy is {accuracy*100:.2f}%, which may or may not be satisfactory depending on the business context. "
                               "Given the positive accuracy value, this model is closer to being business-ready compared to others, but there's still room for improvement.")

        # How far into the future are they accurate?
        if sku in abnormal_skus:
            interpretation += ("\nBased on the forecast's significant discrepancy, it's safe to say that the predictions for this SKU are "
                               "unreliable even for the immediate future.")
        else:
            interpretation += ("\nGiven the positive accuracy value, the forecast seems more reliable for the immediate future, but "
                               "its reliability may decrease for extended horizons.")

        interpretations[sku] = interpretation

    return interpretations

detailed_interpretations = interpret_dynamic_results(ensemble_results)
for sku, text in detailed_interpretations.items():
    print(f"{sku}: {text}\n")

SKU1: For SKU1, the MAPE value is alarmingly high, suggesting the forecast might be producing extreme values. The 'Accuracy' is outside the standard range of 0% to 100%, indicating significant forecast errors. The large discrepancy in the model's forecast suggests potential issues like extreme values, outliers, or a model not suited for this data. Immediate reevaluation is recommended for this SKU.
Based on the forecast's significant discrepancy, it's safe to say that the predictions for this SKU are unreliable even for the immediate future.

SKU2: For SKU2, the MAPE value is alarmingly high, suggesting the forecast might be producing extreme values. The 'Accuracy' is outside the standard range of 0% to 100%, indicating significant forecast errors. The large discrepancy in the model's forecast suggests potential issues like extreme values, outliers, or a model not suited for this data. Immediate reevaluation is recommended for this SKU.
Based on the forecast's significant discrepancy, 